# Overview

Use Recursive Feature Elimination (RFE) to select the most important features. First optimize hyperparameters on the full features, then use the hyperparameters to run RFE. Use SMOTE to balance the data.

In [1]:
import os
import re
import tempfile
import warnings

import japanize_matplotlib
import lightgbm as lgb
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import mlflow
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from hyperopt import STATUS_OK, SparkTrials, Trials, fmin, hp, tpe
from hyperopt.pyll.base import scope
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImblearnPipeline
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import ColSpec, Schema
from pyspark.sql import SparkSession
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import RFE, RFECV
from sklearn.impute import SimpleImputer
from sklearn.metrics import (
    accuracy_score,
    auc,
    confusion_matrix,
    f1_score,
    log_loss,
    precision_score,
    recall_score,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler

from JapanHorseRaceAnalytics.utilities.base import get_base_dir, read_hive_table
from JapanHorseRaceAnalytics.utilities.metrics import (
    calculate_binary_classifier_statistics,
)
from JapanHorseRaceAnalytics.utilities.mlflow import get_colspecs
from JapanHorseRaceAnalytics.utilities.structured_logger import logger

# Set pandas display options
pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 200)

# Set seed for reproducibility
os.environ["PYTHONHASHSEED"] = str(42)
np.random.seed(42)
tf.random.set_seed(42)
random_state = 42

# Prepare the data

In [2]:
warehouse_dir = f"{get_base_dir()}/spark-warehouse"
postgres_driver_path = f"{get_base_dir()}/jars/postgresql-42.7.1.jar"

spark = (
    SparkSession.builder.appName("20240211_competitors")
    .config("spark.driver.memory", "20g")
    .config("spark.sql.warehouse.dir", warehouse_dir)
    .config("spark.jars", postgres_driver_path)
    .config("spark.executor.extraClassPath", postgres_driver_path)
    .config("spark.driver.extraClassPath", postgres_driver_path)
    .enableHiveSupport()
    .getOrCreate()
)

24/02/25 15:13:24 WARN Utils: Your hostname, Hanks-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.40.105 instead (on interface en0)
24/02/25 15:13:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/02/25 15:13:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/25 15:13:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = read_hive_table(
    table_name="features_20240217_v1",
    schema="jhra_curated",
    spark_session=spark,
    # use_cache=False,
)

# drop from data where cat_トラック種別 == "障害"
data = data[(data["cat_トラック種別"] != "障害")]

# This would mess up the number of horses in the race
# No, it wouldn't because the number of horses in the race is already calculated
data = data[(data["meta_int_race_horses_異常区分"] == "0")]

data = data.reset_index(drop=True)
data.head()

{"event": "Read from hive jhra_curated.features_20240217_v1", "level": "info", "timestamp": "2024-02-25T06:13:25.300347Z", "logger": "JapanHorseRaceAnalytics.utilities.base"}
24/02/25 15:13:26 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/02/25 15:13:26 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/02/25 15:13:26 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
{"event": "Write to parquet /Users/hankehly/Projects/JapanHorseRaceAnalytics/data/sql_tables/features_20240217_v1.snappy.parquet", "level": "info", "timestamp": "2024-02-25T06:13:27.719752Z", "logger": "JapanHorseRaceAnalytics.utilities.base"}
24/02/25 15:13:28 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
{"event": "Read from parquet /Users/hankehly/Projects/JapanHorseRaceAnalytics/data/sql_tables/features_2024021

,meta_レースキー,meta_馬番,meta_着順,meta_本賞金,meta_単勝的中,meta_単勝払戻金,meta_複勝的中,meta_複勝払戻金,meta_int_races_レースキー,meta_発走日時,meta_場コード,cat_四半期,cat_距離,cat_事前_馬場状態コード,cat_事前_レース条件_トラック情報_右左,cat_事前_レース条件_トラック情報_内外,cat_事前_レース条件_種別,cat_事前_レース条件_条件,cat_事前_レース条件_記号,cat_事前_レース条件_重量,cat_事前_レース条件_グレード,num_事前_馬場差,num_頭数,cat_トラック種別,cat_馬場状態内,cat_馬場状態中,cat_馬場状態外,num_直線馬場差最内,num_直線馬場差内,num_直線馬場差中,num_直線馬場差外,num_直線馬場差大外,cat_芝種類,cat_草丈,cat_転圧,cat_凍結防止剤,num_中間降水量,meta_int_race_horses_レースキー,meta_int_race_horses_馬番,meta_int_race_horses_血統登録番号,meta_int_race_horses_発走日時,meta_int_race_horses_異常区分,num_事前ＩＤＭ,cat_事前脚質,num_事前単勝オッズ,num_事前複勝オッズ,cat_事前馬体,cat_事前気配コード,cat_事前上昇度,cat_事前クラスコード,num_事前テン指数,num_事前ペース指数,num_事前上がり指数,num_負担重量,num_馬体重,num_馬体重増減,cat_性別,cat_トラック種別瞬発戦好走馬,cat_トラック種別消耗戦好走馬,num_一走前不利,num_二走前不利,num_三走前不利,num_一走前着順,num_二走前着順,num_三走前着順,num_四走前着順,num_五走前着順,num_六走前着順,num_1走前上昇度,num_2走前上昇度,num_3走前上昇度,num_4走前上昇度,num_5走前上昇度,num_騎手指数,num_情報指数,num_オッズ指数,num_パドック指数,num_総合指数,cat_馬具変更情報,cat_脚元情報,cat_見習い区分,cat_オッズ印,cat_パドック印,cat_直前総合印,cat_距離適性,num_ローテーション,num_基準オッズ,num_基準人気順位,num_基準複勝オッズ,num_基準複勝人気順位,num_特定情報◎,num_特定情報○,num_特定情報▲,num_特定情報△,num_特定情報×,num_総合情報◎,num_総合情報○,num_総合情報▲,num_総合情報△,num_総合情報×,...,num_競争相手平均調教師1位完走,num_競争相手調教師1位完走標準偏差,num_競争相手最高調教師トップ3完走,num_競争相手最低調教師トップ3完走,num_競争相手平均調教師トップ3完走,num_競争相手調教師トップ3完走標準偏差,num_競争相手最高調教師1位完走率,num_競争相手最低調教師1位完走率,num_競争相手平均調教師1位完走率,num_競争相手調教師1位完走率標準偏差,num_競争相手最高調教師トップ3完走率,num_競争相手最低調教師トップ3完走率,num_競争相手平均調教師トップ3完走率,num_競争相手調教師トップ3完走率標準偏差,num_競争相手最高調教師場所レース数,num_競争相手最低調教師場所レース数,num_競争相手平均調教師場所レース数,num_競争相手調教師場所レース数標準偏差,num_競争相手最高調教師場所1位完走,num_競争相手最低調教師場所1位完走,num_競争相手平均調教師場所1位完走,num_競争相手調教師場所1位完走標準偏差,num_競争相手最高調教師場所トップ3完走,num_競争相手最低調教師場所トップ3完走,num_競争相手平均調教師場所トップ3完走,num_競争相手調教師場所トップ3完走標準偏差,num_競争相手最高調教師場所1位完走率,num_競争相手最低調教師場所1位完走率,num_競争相手平均調教師場所1位完走率,num_競争相手調教師場所1位完走率標準偏差,num_競争相手最高調教師場所トップ3完走率,num_競争相手最低調教師場所トップ3完走率,num_競争相手平均調教師場所トップ3完走率,num_競争相手調教師場所トップ3完走率標準偏差,num_競争相手最高調教師本賞金累計,num_競争相手最低調教師本賞金累計,num_競争相手平均調教師本賞金累計,num_競争相手調教師本賞金累計標準偏差,num_競争相手最高調教師1位完走平均賞金,num_競争相手最低調教師1位完走平均賞金,num_競争相手平均調教師1位完走平均賞金,num_競争相手調教師1位完走平均賞金標準偏差,num_競争相手最高調教師レース数平均賞金,num_競争相手最低調教師レース数平均賞金,num_競争相手平均調教師レース数平均賞金,num_競争相手調教師レース数平均賞金標準偏差,num_競争相手平均調教師レース数差,num_競争相手平均調教師1位完走差,num_競争相手平均調教師トップ3完走差,num_競争相手平均調教師1位完走率差,num_競争相手平均調教師トップ3完走率差,num_競争相手平均調教師場所レース数差,num_競争相手平均調教師場所1位完走差,num_競争相手平均調教師場所トップ3完走差,num_競争相手平均調教師場所1位完走率差,num_競争相手平均調教師場所トップ3完走率差,num_競争相手平均調教師本賞金累計差,num_競争相手平均調教師1位完走平均賞金差,num_競争相手平均調教師レース数平均賞金差,meta_int_combinations_レースキー,meta_int_combinations_馬番,num_馬騎手レース数,num_馬騎手1位完走,num_馬騎手1位完走率,num_馬騎手トップ3完走,num_馬騎手トップ3完走率,num_馬騎手初二走,num_馬騎手同騎手,num_馬騎手場所レース数,num_馬騎手場所1位完走,num_馬騎手場所1位完走率,num_馬騎手場所トップ3完走,num_馬騎手場所トップ3完走率,num_馬調教師レース数,num_馬調教師1位完走,num_馬調教師1位完走率,num_馬調教師トップ3完走,num_馬調教師トップ3完走率,num_馬調教師初二走,num_馬調教師同調教師,num_馬調教師場所レース数,num_馬調教師場所1位完走,num_馬調教師場所1位完走率,num_馬調教師場所トップ3完走,num_馬調教師場所トップ3完走率,meta_int_race_weather_レースキー,num_temperature,num_precipitation,num_snowfall,num_snow_depth,num_wind_speed,cat_wind_direction,num_solar_radiation,num_local_air_pressure,num_sea_level_air_pressure,num_relative_humidity,num_vapor_pressure,num_dew_point_temperature,cat_weather,num_visibility
0,01011103,04,6.0,0.0,False,0,False,0,01011103,2001-08-04 01:45:00,01,3,1200,20,1,1,12,A3,102,3,None,NaN,16,芝,1,1,1,1,1,0,0,0,None,NaN,False,False,NaN,01011103,04,98102049,2001-08-04 01:45:00,0,36.0,好位差し,11.5,2.9,None,None,3,18,-12.4,-21.1,-10.9,550,476.0,14,牡,True,False,0.0,0.0,0.0,7.0,2.0,7.0,NaN,NaN,NaN,3,3,3,None,None,0.4,0.4,0.0,1.8,38.6,0,0,0,None,4,4,5,4.0,16.8,6,3.4,6,0,0,0,10,0,3,6,8,87,0,...,34.133333,18.575492,204,32,98.066667,44.395896,0.162376,0.027668,0.075334,0.036725,0.403960,0.120735,0.213261,0.076365,47,3,18.733333,12.390677,8,0,2.066667,2.112397,15,0,4.733333,4.464178,0.333333,0.0,0.094180,0.088041,0.5,0.000000,0.207743,0.136352,231606.0,14187.0,65859.266667,53242.331542,1687.439024,567.857143,913.419366,273.792817,458.625743,49.167979,141.408061,101.993179,73.000000,-3.133333,-5.066667,-0.016287,-0.036119,-6.733333,-1.066667,-0.733333,-0.010847,0.12559

In [4]:
spark.stop()

In [4]:
# num_cols = [col for col in data.columns if col.startswith("num_")]
usecols = pd.read_csv("step3.csv").query("rfe_support").reset_index(drop=True)["Unnamed: 0"].tolist()
usecols = [col[5:] for col in usecols]
# remove cat_ from usecols
usecols = [col for col in usecols if not col.startswith("cat_")]
data[usecols].isna().sum().to_frame()

,0
num_競争相手平均騎手レース数,0
num_調教師場所トップ3完走,0
num_位置指数順位,1173
num_競争相手最低事前単勝オッズ,0
num_競争相手調教師レース数標準偏差,0
num_競争相手平均トップ3完走,0
num_騎手指数変動率,0
num_騎手期待単勝率,66113
num_距離トップ3完走率,0
num_競争相手平均三走前不利,104475


# Train/test split

In [62]:
X = data
y = data["meta_複勝的中"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=random_state
)

print(f"X_train: {X_train.shape}")
print(f"X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

X_train: (857061, 1008)
X_test: (214266, 1008)
y_train: (857061,)
y_test: (214266,)


In [63]:
assert len(X_train.filter(regex="実績", axis=1).columns.tolist()) == 0

In [64]:
num_columns = X_train.filter(regex="num_", axis=1).columns.tolist()
cat_columns = X_train.filter(regex="cat_", axis=1).columns.tolist()
ord_columns = X_train.filter(regex="ord_", axis=1).columns.tolist()
meta_columns = X_train.filter(regex="meta_", axis=1).columns.tolist()

print(f"num_columns: {len(num_columns)}, cat_columns: {len(cat_columns)}, ord_columns: {len(ord_columns)}, meta_columns: {len(meta_columns)}")
print(f"Total columns: {len(num_columns) + len(cat_columns) + len(ord_columns) + len(meta_columns)}, X_train.shape[1]: {X_train.shape[1]}")

num_columns: 913, cat_columns: 70, ord_columns: 0, meta_columns: 25
Total columns: 1008, X_train.shape[1]: 1008


In [65]:
from imblearn.over_sampling import SMOTE

numeric_features = X_train.filter(regex="num_", axis=1).columns.tolist()
categorical_features = list(set(X_train.filter(regex="cat_", axis=1).columns.tolist()) - {"cat_距離"})
preprocessor = ColumnTransformer(
    transformers=[
        ("ord", OrdinalEncoder(categories=[[1000, 1150, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 3000, 3200, 3400, 3600]]), ["cat_距離"]),
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore", drop="if_binary"), categorical_features),
    ],
)

pipeline = ImblearnPipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("smote", SMOTE(random_state=random_state, k_neighbors=5)),
        ("classifier", lgb.LGBMClassifier(**params)),
    ]
)

X_train_p = preprocessor.fit_transform(X_train)

# smote = 
SMOTE(random_state=random_state, k_neighbors=5).fit_resample(X_train_p, y_train)
# X_sm, y_sm = smote.fit_resample(X_train, y_train)

ValueError: Input X contains NaN.
SMOTE does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# Define objective function

In [8]:
def create_objective_fn(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    df_payout: pd.DataFrame,
    experiment_name: str,
):
    """
    df_payout should have the same index as *_test and have the following columns:
    * レースキー
    * 馬番
    * 距離
    * 発走日時
    * 年齢
    * 場コード
    * payout - amount won if betting 100 yen.
    """

    def train(params):
        def profit_loss(row, payout_column_name, bet_amount=100):
            if row["pred"] and row["actual"]:
                payout = row[payout_column_name] * (bet_amount / 100)
                return payout - bet_amount
            elif row["pred"] and not row["actual"]:
                return -bet_amount
            else:
                return 0

        mlflow.set_experiment(experiment_name=experiment_name)
        with mlflow.start_run():
            mlflow.log_params(params)

            numeric_features = X_train.filter(regex="num_", axis=1).columns.tolist()
            categorical_features = list(set(X_train.filter(regex="cat_", axis=1).columns.tolist()) - {"cat_距離"})
            preprocessor = ColumnTransformer(
                transformers=[
                    ("ord", OrdinalEncoder(categories=[[1000, 1150, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 3000, 3200, 3400, 3600]]), ["cat_距離"]),
                    ("num", StandardScaler(), numeric_features),
                    ("cat", OneHotEncoder(handle_unknown="ignore", drop="if_binary"), categorical_features),
                ],
            )
            model = Pipeline(
                steps=[
                    ("preprocessor", preprocessor),
                    ("classifier", lgb.LGBMClassifier(**params)),
                ]
            )

            model.fit(X_train, y_train)
            mlflow.sklearn.log_model(sk_model=model, artifact_path="model")

            y_pred_proba = model.predict_proba(X_test)[:, 1]
            y_pred = model.predict(X_test)

            metrics = {
                "loss": log_loss(y_test, y_pred_proba),
                "accuracy": accuracy_score(y_test, y_pred),
                "precision": precision_score(y_test, y_pred),
                "recall": recall_score(y_test, y_pred),
                "f1": f1_score(y_test, y_pred),
                "roc_auc": roc_auc_score(y_test, y_pred),
            }
            mlflow.log_metrics(metrics)

            # Calculate payout rates by group
            results = pd.concat(
                [
                    df_payout,
                    pd.DataFrame(
                        np.c_[y_test, y_pred, y_pred_proba],
                        columns=["actual", "pred", "pred_proba_true"],
                    ),
                ],
                axis=1,
            )
            payout_all = calculate_binary_classifier_statistics(
                results, group_by=None, payout_column_name="payout"
            )
            payout_month = calculate_binary_classifier_statistics(
                results,
                group_by=results["発走日時"].dt.month,
                payout_column_name="payout",
            )
            payout_distance = calculate_binary_classifier_statistics(
                results,
                group_by=pd.cut(x=results["距離"], bins=[0, 1400, 1800, 10000]),
                payout_column_name="payout",
            )
            payout_season = calculate_binary_classifier_statistics(
                results,
                group_by=results["発走日時"].dt.month % 12 // 3,
                payout_column_name="payout",
            )
            payout_year = calculate_binary_classifier_statistics(
                results,
                group_by=results["発走日時"].dt.year,
                payout_column_name="payout",
            )
            payout_age = calculate_binary_classifier_statistics(
                results,
                group_by=pd.cut(results["年齢"], bins=[0, 3, 6, 100]),
                payout_column_name="payout",
            )
            payout_racetrack = calculate_binary_classifier_statistics(
                results, group_by=results["場コード"], payout_column_name="payout"
            )
            payout = (
                pd.concat(
                    [
                        pd.DataFrame(payout_all).T.assign(group="all"),
                        pd.DataFrame(payout_month).T.assign(group="month"),
                        pd.DataFrame(payout_distance).T.assign(group="distance"),
                        pd.DataFrame(payout_season).T.assign(group="season"),
                        pd.DataFrame(payout_year).T.assign(group="year"),
                        pd.DataFrame(payout_age).T.assign(group="horse_age"),
                        pd.DataFrame(payout_racetrack).T.assign(group="racetrack"),
                    ],
                    axis=0,
                )
                .rename_axis(index="part")
                .reset_index()
            )
            # Move "group" and "part" columns to the first position in this dataframe
            payout = payout[
                ["group", "part"]
                + [c for c in payout.columns if c not in ["group", "part"]]
            ]

            # Save payout rates as csv
            with tempfile.NamedTemporaryFile(prefix="payout_rate_", suffix=".csv") as f:
                payout.to_csv(f.name, index=False)
                mlflow.log_artifact(f.name)

            # Log payout rates as metrics
            payout_metrics = {}
            for group_name, group in payout.groupby("group"):
                for i, row in group.iterrows():
                    key = re.sub(r"\W", "_", f"payout_rate_{group_name}_{row['part']}")
                    payout_metrics[key] = row["payout_rate"]
            mlflow.log_metrics(payout_metrics)

            # Suppress UserWarning messages from matplotlib
            warnings.filterwarnings("ignore", category=UserWarning)

            # Plot payout rates by group
            sns.set_theme(style="whitegrid")
            fig, axes = plt.subplots(2, 4, figsize=(20, 10))
            for (group, df), ax in zip(payout.groupby("group"), axes.flatten()):
                sns.barplot(x="part", y="payout_rate", data=df, ax=ax)
                ax.set_title(group)
                ax.set_ylim(0, 150)
                ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
                ax.set_ylabel("payout rate")
                ax.set_xlabel("")
                ax.yaxis.set_major_formatter(ticker.PercentFormatter())
            with tempfile.NamedTemporaryFile(prefix="payout_rate_", suffix=".png") as f:
                plt.tight_layout()
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Plot bank balance over time
            results["profit_loss"] = results.apply(
                profit_loss, args=("payout", 100), axis=1
            )
            daily_profit_loss = results.groupby("発走日時")["profit_loss"].sum()
            bank_balance = daily_profit_loss.cumsum()
            plt.figure(figsize=(10, 10))
            ax = plt.subplot(1, 1, 1)
            ax.plot(bank_balance.index, bank_balance.values)
            ax.set_title("Bank Balance")
            ax.set_xlabel("Date")
            ax.set_ylabel("Bank Balance")
            ax.grid(True)
            ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:,.0f}"))
            with tempfile.NamedTemporaryFile(
                prefix="bank_balance_", suffix=".png"
            ) as f:
                plt.tight_layout()
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Confusion Matrix
            conf_matrix = confusion_matrix(y_test, y_pred)
            _, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
            sns.heatmap(conf_matrix, annot=True, fmt="g", cmap="Blues", ax=ax1)
            ax1.set_xlabel("Predicted")
            ax1.set_ylabel("Actual")
            ax1.set_title("Confusion Matrix")
            sns.heatmap(
                conf_matrix / conf_matrix.sum(axis=1)[:, None],
                annot=True,
                fmt=".2%",
                cmap="Blues",
                ax=ax2,
            )
            ax2.set_xlabel("Predicted")
            ax2.set_ylabel("Actual")
            ax2.set_title("Normalized Confusion Matrix")
            with tempfile.NamedTemporaryFile(prefix="confusion_matrix_", suffix=".png") as f:
                plt.tight_layout()
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # ROC Curve
            fpr, tpr, _ = roc_curve(y_test, y_pred)
            roc_auc = auc(fpr, tpr)
            _, ax = plt.subplots(figsize=(10, 10))
            ax.plot(fpr, tpr, color="darkorange", lw=2, label="ROC curve (area = %0.2f)" % roc_auc)
            ax.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
            ax.set_xlim([0.0, 1.0])
            ax.set_ylim([0.0, 1.0])
            ax.set_xlabel("False Positive Rate")
            ax.set_ylabel("True Positive Rate")
            ax.set_title("Receiver Operating Characteristic")
            ax.legend(loc="lower right")
            with tempfile.NamedTemporaryFile(prefix="roc_curve_", suffix=".png") as f:
                plt.tight_layout()
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            # Feature Importances
            feature_importances = zip(
                model.named_steps["preprocessor"].get_feature_names_out(),
                model.named_steps["classifier"].feature_importances_,
            )
            feature_importances_df = (
                pd.DataFrame(
                    data=feature_importances, columns=["feature", "importance"]
                )
                .sort_values("importance", ascending=False)
                .reset_index(drop=True)
            )
            with tempfile.NamedTemporaryFile(
                prefix="feature_importance_", suffix=".csv"
            ) as f:
                feature_importances_df.to_csv(f.name, index=False)
                mlflow.log_artifact(f.name)
            font_properties = fm.FontProperties(
                fname="/System/Library/Fonts/ヒラギノ角ゴシック W3.ttc"
            )

            sns.set_theme(style="whitegrid")
            plt.figure(figsize=(10, 12))
            ax = sns.barplot(x="importance", y="feature", data=feature_importances_df.iloc[:50])
            ax.set_title("Feature Importances (Top 50)", fontproperties=font_properties)
            ax.set_xlabel("Importance", fontproperties=font_properties)
            ax.set_ylabel("Features", fontproperties=font_properties)
            for label in ax.get_yticklabels():
                label.set_fontproperties(font_properties)
            with tempfile.NamedTemporaryFile(prefix="feature_importance_", suffix=".png") as f:
                plt.tight_layout()
                plt.savefig(f.name)
                plt.close()
                mlflow.log_artifact(f.name)

            return {"status": STATUS_OK, "params": params, "model": model, **metrics}

    return train

# Optimize hyperparameters

In [9]:
space = {
    "boosting_type": hp.choice("boosting_type", ["gbdt", "dart", "goss"]),
    "learning_rate": hp.loguniform("learning_rate", -5, 0),  # between e^-5 and 1
    "n_estimators": scope.int(hp.quniform("n_estimators", 100, 1000, 1)),
    "max_depth": scope.int(hp.quniform("max_depth", 3, 10, 1)),
    "num_leaves": scope.int(hp.quniform("num_leaves", 20, 150, 1)),
    "min_child_samples": scope.int(hp.quniform("min_child_samples", 20, 500, 1)),
    "feature_fraction": hp.uniform("feature_fraction", 0.5, 1.0),
    "lambda_l1": hp.uniform("lambda_l1", 0, 5),
    "lambda_l2": hp.uniform("lambda_l2", 0, 5),
    "min_split_gain": hp.uniform("min_split_gain", 0, 1),
    "min_child_weight": hp.uniform("min_child_weight", 0.001, 10),
    "subsample": hp.uniform("subsample", 0.5, 1),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "reg_alpha": hp.uniform("reg_alpha", 0.0, 1.0),
    "reg_lambda": hp.uniform("reg_lambda", 0.0, 1.0),
    "objective": "binary",
    "class_weight": "balanced",
    "verbose": -1,
    "seed": 80,
}

In [10]:
df_payout_renamed_columns = {
    "meta_レースキー": "レースキー",
    "meta_馬番": "馬番",
    "cat_距離": "距離",
    "meta_発走日時": "発走日時",
    "meta_複勝払戻金": "payout",
    "num_年齢": "年齢",
    "meta_場コード": "場コード",
}

df_payout = (
    data.iloc[X_test.index]
    .reset_index(drop=True)
    .rename(columns=df_payout_renamed_columns)[df_payout_renamed_columns.values()]
)

experiment_name = "20240223_rfe_full_features_lgbm"
if mlflow.get_experiment_by_name(experiment_name) is None:
    mlflow.create_experiment(experiment_name)

fn = create_objective_fn(
    X_train,
    y_train,
    X_test,
    y_test,
    df_payout=df_payout,
    experiment_name=experiment_name,
)

In [11]:
# serial
trials = Trials()
fmin(
    fn=fn,
    space=space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials,
    trials_save_file="trials_20240223_rfe_full_features_lgbm.pkl",
)

# parallel (runs out of memory)
# trials = SparkTrials(parallelism=2, spark_session=spark)
# fmin(fn=fn, space=space, algo=tpe.suggest, max_evals=60, trials=trials)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

build_posterior_wrapper took 0.002956 seconds
TPE using 0 trials
/Users/hankehly/Projects/JapanHorseRaceAnalytics/.venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")

/Users/hankehly/Projects/JapanHorseRaceAnalytics/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [40] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(

/Users/hankehly/Projects/JapanHorseRaceAnalytics/.venv/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:228: UserWarning: Found unknown categories in columns [40] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(

Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before 

  2%|▏         | 1/50 [08:07<6:37:44, 487.03s/trial, best loss: 0.5113984276080934]

build_posterior_wrapper took 0.001754 seconds
TPE using 1/1 trials with best loss 0.511398
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to th

  4%|▍         | 2/50 [12:24<4:41:31, 351.91s/trial, best loss: 0.5113984276080934]

...

build_posterior_wrapper took 0.004994 seconds
TPE using 48/48 trials with best loss 0.477510
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to 

 98%|█████████▊| 49/50 [5:10:19<05:45, 345.99s/trial, best loss: 0.4775099212400654]

build_posterior_wrapper took 0.001621 seconds
TPE using 49/49 trials with best loss 0.477510
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to the appropriate data type before plotting.
Using categorical units to plot a list of strings that are all parsable as floats or dates. If these strings should be plotted as numbers, cast to 

100%|██████████| 50/50 [5:17:47<00:00, 381.35s/trial, best loss: 0.4775099212400654]


{'boosting_type': 0,
 'colsample_bytree': 0.6378156304978417,
 'feature_fraction': 0.9336946785708267,
 'lambda_l1': 2.577098853592653,
 'lambda_l2': 4.954790152997694,
 'learning_rate': 0.1801994182561799,
 'max_depth': 10.0,
 'min_child_samples': 420.0,
 'min_child_weight': 1.9789716420914754,
 'min_split_gain': 0.7210712578836012,
 'n_estimators': 665.0,
 'num_leaves': 140.0,
 'reg_alpha': 0.8021502939893334,
 'reg_lambda': 0.34804973445991183,
 'subsample': 0.6360877718163862}

In [11]:
# load trials from trial.pkl
import pickle
with open("trials_20240223_rfe_full_features_lgbm.pkl", "rb") as f:
    trials = pickle.load(f)

# RFE

In [13]:
best_params = trials.best_trial["result"]["params"]

numeric_features = X_train.filter(regex="num_", axis=1).columns.tolist()
categorical_features = list(set(X_train.filter(regex="cat_", axis=1).columns.tolist()) - {"cat_距離"})

preprocessor = ColumnTransformer(
    transformers=[
        ("ord", OrdinalEncoder(categories=[[1000, 1150, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400, 2500, 2600, 3000, 3200, 3400, 3600]]), ["cat_距離"]),
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore", drop="if_binary"), categorical_features),
    ]
)

X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep = preprocessor.transform(X_test)

## Initial Reduction

Use RFE with a step size of 50-100 to quickly reduce the feature space from 1300 to a few hundred features.

### Round 1

In [20]:
n_features = X_train_prep.shape[1]
print(f"n_features: {n_features}")

n_features: 1386


In [21]:
rfe = RFE(
    estimator=lgb.LGBMClassifier(**best_params),
    step=100,
    verbose=1,
    n_features_to_select=0.5,
)
rfe.fit(X_train_prep, y_train)

Fitting estimator with 1386 features.
Fitting estimator with 1336 features.
Fitting estimator with 1286 features.
Fitting estimator with 1236 features.
Fitting estimator with 1186 features.
Fitting estimator with 1136 features.
Fitting estimator with 1086 features.
Fitting estimator with 1036 features.
Fitting estimator with 986 features.
Fitting estimator with 936 features.
Fitting estimator with 886 features.
Fitting estimator with 836 features.
Fitting estimator with 786 features.
Fitting estimator with 736 features.


RFE(estimator=LGBMClassifier(class_weight='balanced',
                             colsample_bytree=0.6378156304978417,
                             feature_fraction=0.9336946785708267,
                             lambda_l1=2.577098853592653,
                             lambda_l2=4.954790152997694,
                             learning_rate=0.1801994182561799, max_depth=10,
                             min_child_samples=420,
                             min_child_weight=1.9789716420914754,
                             min_split_gain=0.7210712578836012,
                             n_estimators=665, num_leaves=140,
                             objective='binary', reg_alpha=0.8021502939893334,
                             reg_lambda=0.34804973445991183, seed=80,
                             subsample=0.6360877718163862, verbose=-1),
    n_features_to_select=0.5, step=50, verbose=1)

In [26]:
y_pred_proba_1 = rfe.predict_proba(X_test_prep)[:, 1]
y_pred_1 = rfe.predict(X_test_prep)
metrics_1 = {
    "loss": log_loss(y_test, y_pred_proba_1),
    "accuracy": accuracy_score(y_test, y_pred_1),
    "precision": precision_score(y_test, y_pred_1),
    "recall": recall_score(y_test, y_pred_1),
    "f1": f1_score(y_test, y_pred_1),
    "roc_auc": roc_auc_score(y_test, y_pred_1),
}
print(metrics_1)

{'loss': 0.4792115048751, 'accuracy': 0.7585757889725855, 'precision': 0.44834491337722626, 'recall': 0.6579398472806608, 'f1': 0.5332876205599214, 'roc_auc': 0.7216045732060151}


In [33]:
step1 = (
    pd.DataFrame(
        rfe.ranking_, index=preprocessor.get_feature_names_out(), columns=["ranking"]
    )
    .assign(rfe_support=rfe.support_)
    .sort_values("ranking")
)
step1.to_csv("step1.csv", index=True)
step1

,ranking,rfe_support
ord__cat_距離,1,True
num__num_競争相手平均基準複勝オッズ差,1,True
num__num_競争相手平均人気指数差,1,True
num__num_競争相手平均調教指数差,1,True
num__num_競争相手平均厩舎指数差,1,True
...,...,...
cat__cat_事前_レース条件_記号_303,15,False
cat__cat_事前_レース条件_記号_400,15,False
cat__cat_事前_レース条件_記号_401,15,False
cat__cat_事前_レース条件_記号_500,15,False


### Round 2

In [34]:
n_features_2 = step1.query("rfe_support == True").shape[0]
print(f"n_features: {n_features_2}")

n_features: 693


In [35]:
X_train_prep_2 = X_train_prep[:, rfe.support_]
X_test_prep_2 = X_test_prep[:, rfe.support_]

In [37]:
rfe_2 = RFE(
    estimator=lgb.LGBMClassifier(**best_params),
    step=50,
    verbose=1,
    n_features_to_select=0.5,
)
rfe_2.fit(X_train_prep_2, y_train)

Fitting estimator with 693 features.
Fitting estimator with 678 features.
Fitting estimator with 663 features.
Fitting estimator with 648 features.
Fitting estimator with 633 features.
Fitting estimator with 618 features.
Fitting estimator with 603 features.
Fitting estimator with 588 features.
Fitting estimator with 573 features.
Fitting estimator with 558 features.
Fitting estimator with 543 features.
Fitting estimator with 528 features.
Fitting estimator with 513 features.
Fitting estimator with 498 features.
Fitting estimator with 483 features.
Fitting estimator with 468 features.
Fitting estimator with 453 features.
Fitting estimator with 438 features.
Fitting estimator with 423 features.
Fitting estimator with 408 features.
Fitting estimator with 393 features.
Fitting estimator with 378 features.
Fitting estimator with 363 features.
Fitting estimator with 348 features.


RFE(estimator=LGBMClassifier(class_weight='balanced',
                             colsample_bytree=0.6378156304978417,
                             feature_fraction=0.9336946785708267,
                             lambda_l1=2.577098853592653,
                             lambda_l2=4.954790152997694,
                             learning_rate=0.1801994182561799, max_depth=10,
                             min_child_samples=420,
                             min_child_weight=1.9789716420914754,
                             min_split_gain=0.7210712578836012,
                             n_estimators=665, num_leaves=140,
                             objective='binary', reg_alpha=0.8021502939893334,
                             reg_lambda=0.34804973445991183, seed=80,
                             subsample=0.6360877718163862, verbose=-1),
    n_features_to_select=0.5, step=15, verbose=1)

In [40]:
y_pred_proba_2 = rfe_2.predict_proba(X_test_prep_2)[:, 1]
y_pred_2 = rfe_2.predict(X_test_prep_2)
metrics_2 = {
    "loss": log_loss(y_test, y_pred_proba_2),
    "accuracy": accuracy_score(y_test, y_pred_2),
    "precision": precision_score(y_test, y_pred_2),
    "recall": recall_score(y_test, y_pred_2),
    "f1": f1_score(y_test, y_pred_2),
    "roc_auc": roc_auc_score(y_test, y_pred_2),
}
print(metrics_2)

{'loss': 0.4795363134554545, 'accuracy': 0.7567556215171796, 'precision': 0.4459215863001352, 'recall': 0.66085620784078, 'f1': 0.5325189032101822, 'roc_auc': 0.7215244918103437}


In [45]:
step2 = (
    pd.DataFrame(rfe_2.ranking_, index=step1[step1.rfe_support].index, columns=["ranking"])
    .assign(rfe_support=rfe_2.support_)
    .sort_values("ranking")
)
step2.to_csv("step2.csv", index=True)
step2

,ranking,rfe_support
num__num_競争相手最高入厩何日前,1,True
num__num_基準オッズ,1,True
num__num_競争相手騎手場所レース数標準偏差,1,True
num__num_競争相手平均騎手場所レース数,1,True
num__num_競争相手騎手過去5走トップ3完走率標準偏差,1,True
...,...,...
num__num_競争相手平均1着単勝オッズ変動率差,25,False
num__num_競争相手平均一走前着順,25,False
num__num_1着平均馬体重差,25,False
num__num_競争相手最小騎手場所トップ3完走,25,False


## Fine-tuning
Once you have a reduced set of features (e.g., 200-300), use a smaller step size (10-20) for more detailed feature elimination, allowing the model to assess the importance of features more accurately.

### Round 3

In [46]:
n_features_3 = step2.query("rfe_support == True").shape[0]
print(f"n_features: {n_features_3}")

n_features: 346


In [47]:
X_train_prep_3 = X_train_prep_2[:, rfe_2.support_]
X_test_prep_3 = X_test_prep_2[:, rfe_2.support_]

In [49]:
rfe_3 = RFE(
    estimator=lgb.LGBMClassifier(**best_params),
    step=15,
    verbose=1,
    n_features_to_select=130,
)
rfe_3.fit(X_train_prep_3, y_train)

Fitting estimator with 346 features.
Fitting estimator with 331 features.
Fitting estimator with 316 features.
Fitting estimator with 301 features.
Fitting estimator with 286 features.
Fitting estimator with 271 features.
Fitting estimator with 256 features.
Fitting estimator with 241 features.
Fitting estimator with 226 features.
Fitting estimator with 211 features.
Fitting estimator with 196 features.
Fitting estimator with 181 features.
Fitting estimator with 166 features.
Fitting estimator with 151 features.
Fitting estimator with 136 features.


RFE(estimator=LGBMClassifier(class_weight='balanced',
                             colsample_bytree=0.6378156304978417,
                             feature_fraction=0.9336946785708267,
                             lambda_l1=2.577098853592653,
                             lambda_l2=4.954790152997694,
                             learning_rate=0.1801994182561799, max_depth=10,
                             min_child_samples=420,
                             min_child_weight=1.9789716420914754,
                             min_split_gain=0.7210712578836012,
                             n_estimators=665, num_leaves=140,
                             objective='binary', reg_alpha=0.8021502939893334,
                             reg_lambda=0.34804973445991183, seed=80,
                             subsample=0.6360877718163862, verbose=-1),
    n_features_to_select=130, step=15, verbose=1)

In [50]:
y_pred_proba_3 = rfe_3.predict_proba(X_test_prep_3)[:, 1]
y_pred_3 = rfe_3.predict(X_test_prep_3)
metrics_3 = {
    "loss": log_loss(y_test, y_pred_proba_3),
    "accuracy": accuracy_score(y_test, y_pred_3),
    "precision": precision_score(y_test, y_pred_3),
    "recall": recall_score(y_test, y_pred_3),
    "f1": f1_score(y_test, y_pred_3),
    "roc_auc": roc_auc_score(y_test, y_pred_3),
}
print(metrics_3)

{'loss': 0.488040630920827, 'accuracy': 0.7499789980678222, 'precision': 0.4388672206206546, 'recall': 0.6913778133974487, 'f1': 0.5369155364228106, 'roc_auc': 0.7284503373706583}


In [51]:
step3 = (
    pd.DataFrame(rfe_3.ranking_, index=step2[step2.rfe_support].index, columns=["ranking"])
    .assign(rfe_support=rfe_3.support_)
    .sort_values("ranking")
)
step3.to_csv("step3.csv", index=True)
step3

,ranking,rfe_support
num__num_競争相手平均騎手レース数,1,True
num__num_調教師場所トップ3完走,1,True
num__num_位置指数順位,1,True
num__num_競争相手最低事前単勝オッズ,1,True
num__num_競争相手調教師レース数標準偏差,1,True
...,...,...
num__num_競争相手平均総合指数,16,False
num__num_競争相手最高パドック指数,16,False
num__num_競争相手平均本賞金累計,16,False
num__num_競争相手入厩何日前標準偏差,16,False


### Round 4

In [52]:
n_features_4 = step3.query("rfe_support == True").shape[0]
print(f"n_features: {n_features_4}")

n_features: 130


In [53]:
X_train_prep_4 = X_train_prep_3[:, rfe_3.support_]
X_test_prep_4 = X_test_prep_3[:, rfe_3.support_]

In [57]:
rfe_4 = RFECV(
    estimator=lgb.LGBMClassifier(**best_params),
    step=5,
    min_features_to_select=20,
    cv=5,
    n_jobs=-1,
    verbose=1,
)
rfe_4.fit(X_train_prep_4, y_train)

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


Fitting estimator with 130 features.
Fitting estimator with 130 features.
Fitting estimator with 130 features.
Fitting estimator with 130 features.
Fitting estimator with 130 features.
Fitting estimator with 125 features.
Fitting estimator with 125 features.
Fitting estimator with 125 features.
Fitting estimator with 125 features.
Fitting estimator with 125 features.
Fitting estimator with 120 features.
Fitting estimator with 120 features.
Fitting estimator with 120 features.
Fitting estimator with 120 features.
Fitting estimator with 120 features.
Fitting estimator with 115 features.
Fitting estimator with 115 features.
Fitting estimator with 115 features.
Fitting estimator with 115 features.
Fitting estimator with 115 features.
Fitting estimator with 110 features.
Fitting estimator with 110 features.
Fitting estimator with 110 features.
Fitting estimator with 110 features.
Fitting estimator with 110 features.
Fitting estimator with 105 features.
Fitting estimator with 105 features.
F

KeyboardInterrupt: 

In [ ]:
y_pred_proba_4 = rfe_4.predict_proba(X_test_prep_4)[:, 1]
y_pred_4 = rfe_4.predict(X_test_prep_4)
metrics_4 = {
    "loss": log_loss(y_test, y_pred_proba_4),
    "accuracy": accuracy_score(y_test, y_pred_4),
    "precision": precision_score(y_test, y_pred_4),
    "recall": recall_score(y_test, y_pred_4),
    "f1": f1_score(y_test, y_pred_4),
    "roc_auc": roc_auc_score(y_test, y_pred_4),
}
print(metrics_4)

{'loss': 0.5221410055247936, 'accuracy': 0.7247066730139173, 'precision': 0.41374701085290333, 'recall': 0.7511075491440148, 'f1': 0.5335747722672065, 'roc_auc': 0.7344057176238478}


## Final Model Training
After selecting the final set of features, perform a thorough hyperparameter optimization to fine-tune your model.